In [ ]:
import pandas as pd
# Imports
import pandas as pd
import numpy as np
#from keras.models import Sequential
#from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import KernelPCA



from sklearn.model_selection import KFold
#from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


#from imblearn.over_sampling import SMOTE

# Load the data from CSV files into DataFrames
zone_data_1A = pd.read_csv('../[3.1] - OUTLIER_REMOVAL/zone_data_1A_MONTHLY.csv')
zone_data_7 = pd.read_csv('../[3.1] - OUTLIER_REMOVAL/zone_data_7_MONTHLY.csv')
zone_data_6 = pd.read_csv('../[3.1] - OUTLIER_REMOVAL/zone_data_6_MONTHLY.csv')


# Copy the data to avoid SettingWithCopyWarning
zone_data_1A_top_sorted = zone_data_1A.copy()
#zone_data_1A_bottom_sorted = zone_data_1A_bottom.copy()

# Copy the data for zone 6 and zone 7 to avoid SettingWithCopyWarning
zone_data_6_top_sorted = zone_data_6.copy()
#zone_data_6_bottom_sorted = zone_data_6.copy()

zone_data_7_top_sorted = zone_data_7.copy()
#zone_data_7_bottom_sorted = zone_data_7.copy()

# Order the data by 'Create_date_ordinal' from lowest to highest
zone_data_1A_top_sorted.sort_values(by='date', ascending=True, inplace=True)
#zone_data_1A_bottom_sorted.sort_values(by='Create_date_ordinal', ascending=True, inplace=True)

# Order the data by 'Create_date_ordinal' from lowest to highest
zone_data_6_top_sorted.sort_values(by='date', ascending=True, inplace=True)
#zone_data_6_bottom_sorted.sort_values(by='Create_date_ordinal', ascending=True, inplace=True)

zone_data_7_top_sorted.sort_values(by='date', ascending=True, inplace=True)
#zone_data_7_bottom_sorted.sort_values(by='Create_date_ordinal', ascending=True, inplace=True)






In [ ]:
import itertools

def determine_arma_order(series_data, p_range, q_range):
    """
    Determine the best ARMA model order based on AIC.
    
    Parameters:
        series_data (pd.Series): The time series data.
        p_range (int): The range of AR terms to test.
        q_range (int): The range of MA terms to test.
    
    Returns:
        tuple: The best order (p, q) based on the lowest AIC.
    """
    best_aic = float('inf')
    best_order = None
    
    for p, q in itertools.product(range(p_range), range(q_range)):
        try:
            model = ARMA(series_data, order=(p, 0, q)).fit(disp=0)
            aic = model.aic
            if aic < best_aic:
                best_aic = aic
                best_order = (p, 0, q)
        except Exception as e:
            continue
    
    return best_order

# Example usage within the loop
for i, (zone_name, zone_data) in enumerate(zones.items()):
    zone_data['date'] = pd.to_datetime(zone_data['date'])
    series_data = zone_data.set_index('date')['mean_price']
    
    # Differencing the data to make it stationary
    differenced_data = series_data.diff().dropna()
    
    # Determine the best order for the ARMA model
    best_order = determine_arma_order(differenced_data, p_range=11, q_range=11)
    
    train_data, test_data = train_test_split(differenced_data, test_size=0.1, shuffle=False)
    
    arma_model = build_arma_model(train_data, order=best_order)
    forecast_series, conf_df = forecast_and_simulate(arma_model, steps=len(test_data))
    
    # Rest of the code remains the same